In [1]:
from process.clinic_data import writeClinic
from process.img_data import imgRedir
from process.pat_data import combinePatData, genHCData
from src.feature.base import genROI
from src.utils.data import getPandas
import os
os.chdir('..')

In [2]:
os.getcwd()

'/home/biobot/disk/ldopa'

In [42]:
pat = getPandas('hc_data')

In [7]:
import nibabel as nib
tpm = nib.concat_images([
    'data/bin/PD25-GM-tissuemap-1mm.nii',
    'data/bin/PD25-WM-tissuemap-1mm.nii',
    'data/bin/PD25-CSF-tissuemap-1mm.nii'
])
nib.save(tpm, 'data/bin/PD25-tissuemap-1mm.nii')

In [6]:
from nipype.pipeline.engine import Workflow, Node
from nipype import Function
import nipype.interfaces.fsl as fsl
fsl.FSLCommand.set_default_output_type('NIFTI')
import nipype.interfaces.utility as util
from nipype.interfaces.cat12.preprocess import CAT12Segment
import nipype.interfaces.io as nio

In [43]:
pat.iloc[0]['IMG_ROOT']

'data/subj/101747BLI1495885'

In [39]:
wf = Workflow(name='test', base_dir=os.path.abspath('tmp'))
seg = Node(CAT12Segment(), name='seg')
seg.inputs.in_files = os.path.abspath(pat.iloc[0]['IMG_ROOT']+'/raw/raw.nii')
sink = Node(nio.DataSink(), name='sink')
sink.inputs.base_directory = os.path.abspath(os.path.join('data', 'subj'))
sink.inputs.container = pat.iloc[0]['KEY']
sink.inputs.parameterization = False
wf.connect([
    (seg, sink, [
        ('gm_modulated_image', 'cat12.mri.@gm'),
        ('wm_modulated_image', 'cat12.mri.@wm'),
        ('csf_modulated_image', 'cat12.mri.@csf'),
        ('mri_images', 'cat12.mri.@mri_images'),
        ('label_files', 'cat12.label.@label_files'),
        ('label_roi', 'cat12.label.@label_roi'),
        ('label_rois', 'cat12.label.@label_rois'),
        ('lh_central_surface', 'cat12.surf.@lh_central_surface'),
        ('lh_sphere_surface', 'cat12.surf.@lh_sphere_surface'),
        ('rh_central_surface', 'cat12.surf.@rh_central_surface'),
        ('rh_sphere_surface', 'cat12.surf.@rh_sphere_surface'),
        ('report_files', 'cat12.report.@report_files'),
        ('report', 'cat12.report.@report'),
        ('surface_files', 'cat12.surf.@surface_files'),
        ]),
])

In [40]:
wf.run()

230402-18:20:50,664 nipype.workflow INFO:
	 Workflow test settings: ['check', 'execution', 'logging', 'monitoring']
230402-18:20:50,673 nipype.workflow INFO:
	 Running serially.
230402-18:20:50,674 nipype.workflow INFO:
	 [Node] Setting-up "test.seg" in "/home/biobot/disk/ldopa/tmp/test/seg".
230402-18:20:50,682 nipype.workflow INFO:
	 [Node] Cached "test.seg" - collecting precomputed outputs
230402-18:20:50,684 nipype.workflow INFO:
	 [Node] "test.seg" found cached.
230402-18:20:50,684 nipype.workflow INFO:
	 [Node] Setting-up "test.sink" in "/home/biobot/disk/ldopa/tmp/test/sink".
230402-18:20:50,695 nipype.workflow INFO:
	 [Node] Outdated cache found for "test.sink".
230402-18:20:50,702 nipype.workflow INFO:
	 [Node] Executing "sink" <nipype.interfaces.io.DataSink>
230402-18:20:50,712 nipype.workflow INFO:
	 [Node] Finished "sink", elapsed time 0.007499s.
